In [1]:
from reinforced_visual_slam.srv import *
import rospy

import matplotlib.pyplot as plt
from matplotlib import colors

import os
import sys
sys.path.insert(0,'/misc/software/opencv/opencv-3.2.0_cuda8_with_contrib-x86_64-gcc5.4.0/lib/python3.5/dist-packages')

import cv2
import numpy as np
from skimage.transform import resize

import tensorflow as tf

sys.path.insert(0,'/misc/lmbraid19/thomasa/catkin_ws/src/reinforced_visual_slam/networks/depth_fusion')
from net.my_models import *

import random

import lmbspecialops as sops

/misc/lmbraid19/thomasa/virtual-env/tf_venv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Using /misc/lmbraid19/thomasa/deep-networks/lmbspecialops/build/lib/lmbspecialops.so


# check sparse invariant gradients #

In [2]:
# define the sig params to be checked!
sig_params_list = [{'deltas':[1,], 'weights':[1,], 'epsilon': 1e-9},
                   {'deltas':[2,], 'weights':[1,], 'epsilon': 1e-9},
                   {'deltas':[4,], 'weights':[1,], 'epsilon': 1e-9},
                   ]

# helper function to invert gt depth
def invert_finite_depth(x):
    # mask is true if input is finite and greater than 0. If condition is false, make it invalid (nan)
    mask = tf.logical_and(tf.is_finite(x), tf.greater(x, 0.))
    ix_clean = tf.where(mask, tf.reciprocal(x), tf.fill(x.shape, np.nan))
    return ix_clean


In [ ]:
# load data file
test_file = '/misc/lmbraid19/thomasa/datasets/LSDDepthTraining/test/rgbd_lsdDepth_test.txt'
base_path = os.path.dirname(test_file)
sess = tf.Session()
with open(test_file) as file:
    # read lines of file and loop over it randomly
    lines = file.readlines()
    random.shuffle(lines)
    cnt = 0
    for line in lines:
        # load only the depth GT image, convert to tensor and convert scale to meters
        depth_gt_file = line.strip().split(',')[0]
        depth_gt = tf.read_file(os.path.join(base_path, depth_gt_file))
        depth_tensor = tf.image.decode_png(depth_gt, dtype=tf.uint16)
        depth_tensor = tf.reshape(depth_tensor, [1, 480, 640, 1])
        depth_tensor = tf.cast(depth_tensor, tf.float32)
        depth_tensor =  tf.scalar_mul(0.0002, depth_tensor)
        # make inverse depth similar to training input and downsample it
        idepth_tensor = invert_finite_depth(depth_tensor)
        idepth_tensor = tf.image.resize_images(idepth_tensor, [240, 320], align_corners=True, 
                                               method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
        idepth_tensor = tf.transpose(idepth_tensor,[0,3,1,2])
        # find different scale invariant gradients and plot it alongside the depth gt
        sig_list = []
        for sig_param in sig_params_list:
            gt_sig = sops.scale_invariant_gradient(idepth_tensor,**sig_param)
            sig_list.append(sops.replace_nonfinite(tf.transpose(gt_sig, [0,2,3,1])))        
        sigs_nhwc_x = tf.concat([sig_list[0][:,:,:,0:1], sig_list[1][:,:,:,0:1], 
                                 sig_list[2][:,:,:,0:1]], 0)
        sigs_nhwc_y = tf.concat([sig_list[0][:,:,:,1:], sig_list[1][:,:,:,1:], 
                                 sig_list[2][:,:,:,1:]], 0)
        sigs_nhwc_x = sops.replace_nonfinite(sigs_nhwc_x)
        sigs_nhwc_y = sops.replace_nonfinite(sigs_nhwc_y)
        print('sigs_nhwc_x.shape', sigs_nhwc_x.shape)
        print('sigs_nhwc_y.shape', sigs_nhwc_y.shape)
        tf.summary.image('sigs_x', sigs_nhwc_x, max_outputs=3)
        tf.summary.image('sigs_y', sigs_nhwc_y, max_outputs=3)
        merged = tf.summary.merge_all()
        writer = tf.summary.FileWriter('/tmp/akhil/')
        summary = sess.run(merged)
        writer.add_summary(summary, cnt)
        cnt += 1
        input("Press Enter to continue...")

sigs_nhwc_x.shape (3, 240, 320, 1)
sigs_nhwc_y.shape (3, 240, 320, 1)
Press Enter to continue...
sigs_nhwc_x.shape (3, 240, 320, 1)
sigs_nhwc_y.shape (3, 240, 320, 1)
Press Enter to continue...
sigs_nhwc_x.shape (3, 240, 320, 1)
sigs_nhwc_y.shape (3, 240, 320, 1)
Press Enter to continue...
sigs_nhwc_x.shape (3, 240, 320, 1)
sigs_nhwc_y.shape (3, 240, 320, 1)
Press Enter to continue...
sigs_nhwc_x.shape (3, 240, 320, 1)
sigs_nhwc_y.shape (3, 240, 320, 1)
Press Enter to continue...
sigs_nhwc_x.shape (3, 240, 320, 1)
sigs_nhwc_y.shape (3, 240, 320, 1)
